# Notes:
+ pickle pipeline
+ save output

# Library

In [1]:
import gzip
import shutil
import pandas as pd
from pyarrow.parquet import read_table
from scipy.sparse import coo_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from memory_profiler import memory_usage
import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.cluster import HDBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from scipy import stats
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.cluster import HDBSCAN

import os
import numpy as np
import time
from sklearn.cluster import Birch

import my_utils


In [23]:
import scipy.sparse as sparse

In [10]:
def unzip(input, output):
    with gzip.open(input, 'rb') as f_in, open(output, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print(f'{input} has been successfully uncompressed to {output}.')

def load_data(data_path, row_info_path, column_info_path):
    # Load non_zero parquet data
    table = read_table(data_path)
    nonzero_data = table.to_pandas()
    
    # Adjust column indices to be 0-based
    nonzero_data['col_indices'] = nonzero_data['col_indices'] - 1
    
    # Load row and column index info
    rows = pd.read_csv(row_info_path)
    row_names = rows.iloc[:, 1].to_list()
    
    columns = pd.read_csv(column_info_path)
    column_names = columns.iloc[:, 1].to_list()
    
    # Convert the sparse matrix to a dense DataFrame
    sparse_matrix = coo_matrix(
        (nonzero_data['nonzero_elements'], (nonzero_data['row_indices'], nonzero_data['col_indices'])),
        shape=(len(row_names), len(column_names))
    )
    
    
   

    print('Returning sparse_matrix, column_names, and row_names')
    
    return sparse_matrix, column_names, row_names


class SparseTrainTestSplit(BaseEstimator, TransformerMixin):
    def __init__(self, test_size=0.2, random_state=None):
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y=None):
        # Not needed for this class
        return self

    def transform(self, X):
        # Split the data using train_test_split
        sparse_train, sparse_test, train_indices, test_indices = train_test_split(
            X, row_indices, test_size=self.test_size, random_state=self.random_state
        )

        # Convert the split data back to coo_matrix
        sparse_train = coo_matrix(sparse_train)
        sparse_test = coo_matrix(sparse_test)

        return sparse_train, sparse_test, train_indices, test_indices


In [11]:
class DataClean:
    def __init__(self, remove_by_column=True):
        self.remove_by_column = remove_by_column
        self.output_folder = "../output"
        
        # Create the output folder if it doesn't exist
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

    def fit(self, X, y=None):
        # Check if there are NaN values in the data
        self.has_nan_values = np.isnan(X.data).any()

        with open(os.path.join(self.output_folder, "na_info.txt"), "w") as f:
            if self.has_nan_values:
                # Find rows and columns with NaN values
                self.rows_with_nan = self.find_rows_with_nan(X)
                self.columns_with_nan = self.find_columns_with_nan(X)

                f.write("Rows with NaN Values:\n")
                f.write(", ".join(map(str, self.rows_with_nan)))
                f.write("\nTotal Rows with NaN Values: {}\n".format(len(self.rows_with_nan)))

                f.write("\nColumns with NaN Values:\n")
                f.write(", ".join(map(str, self.columns_with_nan)))
                f.write("\nTotal Columns with NaN Values: {}\n".format(len(self.columns_with_nan)))
            else:
                f.write("No NaN Values Found in the Data.\n")

        return self

    def transform(self, X):
        if self.has_nan_values:
            if self.remove_by_column:
                X = self._remove_nan_by_column(X)
            else:
                X = self._remove_nan_by_row(X)

        return X

    def _remove_nan_by_column(self, X):
        nan_mask = np.isnan(X.data)
        valid_columns = np.unique(X.col[~nan_mask])
        X = X.tocsc()[:, valid_columns].tocoo()
        return X

    def _remove_nan_by_row(self, X):
        if not isinstance(X, coo_matrix):
            raise ValueError("Input must be a sparse COO matrix.")
    
        non_nan_rows = ~np.isnan(X.sum(axis=1).A.ravel())
        X = X.tocsr()[non_nan_rows].tocoo()
        return X

    def find_rows_with_nan(self, X):
        nan_mask = np.isnan(X.data)
        rows_with_nan = np.unique(X.row[nan_mask])
        return rows_with_nan

    def find_columns_with_nan(self, X):
        nan_mask = np.isnan(X.data)
        columns_with_nan = np.unique(X.col[nan_mask])
        return columns_with_nan


In [12]:
def reindex(df, indices, columns, names):
    row_names_indices = [names[i] for i in indices]
    df_with_indices = pd.DataFrame(df, columns=columns, index=row_names_indices)
    return df_with_indices


In [13]:
class DataEDAPCA:
    def __init__(self, columns, z_threshold=10):
        self.columns = columns
        self.z_threshold = z_threshold
        self.removed_indices = None

    def plot_box_and_scatter(self, X):
        # Create a boxplot of the specified columns
        X[self.columns].boxplot()
        plt.title("Boxplot of Selected Columns")
        plt.show()

        # Create a scatterplot of the specified columns
        plt.scatter(X[self.columns[0]], X[self.columns[1]])
        plt.xlabel(self.columns[0])
        plt.ylabel(self.columns[1])
        plt.title("Scatterplot of Selected Columns")
        plt.show()

    def remove_outliers(self, X):
        # Calculate Z-scores for the specified columns
        z_scores = np.abs(stats.zscore(X[self.columns]))
        z_scores = z_scores.to_numpy()  # Convert to a NumPy array
        print("Z-scores:")
        print(z_scores)
        print("Z-threshold:", self.z_threshold)

        # Find the indices of rows with Z-scores exceeding the threshold
        outlier_indices = np.argwhere(z_scores > self.z_threshold)
        outlier_indices = outlier_indices.ravel()
        print("Outlier Indices:", outlier_indices)

        # Drop the rows with high Z-scores
        X.drop(X.index[outlier_indices], inplace=True)
        self.removed_indices = outlier_indices

    def fit_transform(self, X, another_df=None):
        print("Shape before:", X.shape)

        # Step 1: Display initial boxplot and scatterplot
        self.plot_box_and_scatter(X)

        # Step 2: Notify rows with high Z-scores and remove them
        self.remove_outliers(X)

        # Step 3: If another_df is provided, subset it using the same indices
        removed_metadata = None
        if another_df is not None:
            removed_metadata = another_df.iloc[self.removed_indices]

        # Step 4: Display boxplot and scatterplot of the updated data
        self.plot_box_and_scatter(X)

        print("Shape after:", X.shape)

        return X, removed_metadata

In [14]:
def optimize_and_compare_kmeans(data, kmeans_params, alpha=0.05):
    # Perform Grid Search
    grid = GridSearchCV(KMeans(), kmeans_params, cv=3, refit=True)
    grid.fit(data)
    grid_search_estimator = grid.best_estimator_

    # Calculate silhouette scores
    default_kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(data)
    default_silhouette_score = silhouette_score(data, default_kmeans.labels_)
    grid_search_silhouette_score = silhouette_score(data, grid_search_estimator.labels_)

    # Perform a two-sample t-test only if Grid Search performs better
    if grid_search_silhouette_score > default_silhouette_score:
        t_stat, p_value = stats.ttest_ind(default_kmeans.labels_, grid_search_estimator.labels_)

        # Set the default choice to "Grid Search Estimator"
        choice = "KMeans Estimator"

        # Output informative print statements
        print("Default KMeans Silhouette Score:", default_silhouette_score)
        print("Grid Search Estimator Silhouette Score:", grid_search_silhouette_score)

        if p_value < alpha:
            choice = "Grid Search Estimator"
            print("The difference between the two groups is statistically significant.")
            print(f"Using {choice} as it performs significantly better using a threshold of alpha = .05 .")
            return choice, grid_search_estimator
        else:
            print("The difference between the two groups is not statistically significant.")
            print(f"Using {choice} as there is no significant improvement using a threshold of alpha = .05.")
            return choice, default_kmeans
    else:
        choice = "Default Parameter"
        print("Grid Search Estimator Silhouette Score:", grid_search_silhouette_score)
        print("Default KMeans Silhouette Score:", default_silhouette_score)
        print("Default Parameter has a higher Silhouette Score.")
        print("Using Default Parameter as it performs better based on Silhouette Score.")
        return choice, default_kmeans

     # Return both choice and the chosen KMeans clusterer

In [15]:
def find_outliers_in_clusters(data, clusterer, n_neighbors=20, contamination=0.1):
    # Fit the Local Outlier Factor (LOF) model on the data
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    outliers = lof.fit_predict(data)

    # Create a DataFrame to store cluster labels and outlier labels
    cluster_outliers = pd.DataFrame({'Cluster': clusterer.labels_, 'Outlier': outliers})

    # Print the number of data points and percentage of outliers in each cluster
    cluster_info = cluster_outliers.groupby('Cluster').agg(DataPoints=('Cluster', 'count'),
                                                          PercentageOutliers=('Outlier', lambda x: (x == -1).mean() * 100))
    print("Cluster Information:")
    print(cluster_info)

    return cluster_outliers

In [17]:
def label_plot(df, label, cluster_name):
    # Set up the figure
    plt.figure(figsize=(8, 6))

    # Create a scatter plot to visualize the clustering
    sns.scatterplot(x='PC1', y='PC2', data=df, hue=label, palette='viridis', s=50, alpha=0.7)

    # Add labels and title
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Clustering Visualization for' + cluster_name)

    # Show the plot
    plt.grid(True)
    plt.legend(title='Cluster', loc='upper right')
    plt.show()

In [16]:
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    if len(set(labels)) == 1:
        return 0  # Silhouette score is undefined for a single cluster
    return silhouette_score(X, labels)

def optimize_and_compare_hdbscan(data, hdbscan_params, alpha=0.05):
    # Perform Grid Search
    grid_search = GridSearchCV(
        estimator=HDBSCAN(min_cluster_size=20),
        param_grid=hdbscan_params,
        scoring=silhouette_scorer,
        cv=5,
        n_jobs=-1,
    )
    grid_search.fit(data)
    grid_search_estimator = grid_search.best_estimator_

    # Calculate silhouette scores for the default and grid search estimators
    default_hdbscan = HDBSCAN(min_cluster_size=20).fit(data)
    default_labels = default_hdbscan.labels_
    default_silhouette_score = silhouette_score(data, default_labels)

    grid_search_labels = grid_search_estimator.fit_predict(data)
    grid_search_silhouette_score = silhouette_score(data, grid_search_labels)

    # Check if the grid search estimator has a higher silhouette score
    if grid_search_silhouette_score > default_silhouette_score:
        # Perform a two-sample t-test
        t_stat, p_value = stats.ttest_ind(default_labels, grid_search_labels)

        # Check if the p-value is less than the significance level
        if p_value < alpha:
            choice = "Grid Search Estimator"
        else:
            choice = "Default Parameter"
    else:
        choice = "Default Parameter"

    # Output informative print statements
    print("Default HDBSCAN Silhouette Score:", default_silhouette_score)
    print("Grid Search Estimator Silhouette Score:", grid_search_silhouette_score)

    if grid_search_silhouette_score > default_silhouette_score:
        if p_value < alpha:
            print("The difference between the two groups is statistically significant.")
            print(f"Using {choice} as it performs significantly better.")
        else:
            print("The difference between the two groups is not statistically significant.")
            print(f"Using {choice} as there is no significant improvement.")
    else:
        print("Default Parameter has a higher silhouette score. No t-test performed.")

    return choice

# Get data

Unzip .gz file

In [ ]:
# unzip file
unzip('GSE185948_count_RNA.rds.gz', 'data_for_r.rds')
unzip('GSE185948_metadata_RNA.csv.gz', 'uncompressed_metadata.csv')

R code to read RDS file and convert to a paquet file

In [ ]:
filename = file.choose()
data = readRDS(filename)
data
sparse_matrix <- data

num_columns <- ncol(sparse_matrix)
# Get the nonzero elements and their indices
nonzero_elements <- sparse_matrix@x
row_indices <- sparse_matrix@i
col_indices <- rep(1:num_columns, times = diff(sparse_matrix@p))  # Use sparse_matrix@j for column indices


nonzero_data <- data.frame(row_indices, col_indices, nonzero_elements)

col_names <- data.frame(sparse_matrix@Dimnames[2])
nrow(col_names)
row_names <- data.frame((sparse_matrix@Dimnames[1]))

write.csv(row_names, 'row_names.csv')
write.csv(col_names, 'col_names.csv')


# Save the i, j information to a Parquet file
write_parquet(nonzero_data, "non_zero.parquet")

Read csv and paquet data file

In [5]:
# read metadata csv file
metadata_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\GSE185948_metadata_RNA.csv'
metadata = pd.read_csv(metadata_path)

In [19]:
# read rna file from paquet
data_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\non_zero.parquet'
row_info_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\row_names.csv'
column_info_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\col_names.csv'
sparse_matrix, row_names, column_names = load_data(data_path, row_info_path, column_info_path)
row_indices = np.arange(sparse_matrix.shape[0])

Returning sparse_matrix, column_names, and row_names


In [25]:
# convert to csr for faster computation
sparse_matrix = sparse_matrix.tocsr(copy=False)

In [55]:
# toy case
sparse_matrix = sparse_matrix[0:100, 0:50]
row_indices = np.arange(sparse_matrix.shape[0])
row_names = row_names[0:100]
column_names = column_names[0:50]

# Task 1

## Preprocess

In [45]:
split_pipeline = Pipeline([
    ('splitter', SparseTrainTestSplit(test_size=0.2, random_state=42)),
    # Add other steps in the pipeline as needed
])

clean_and_pca_pipeline = Pipeline([
    ('cleaner', DataClean()),  # CleanData is performed first
    ('scaler', StandardScaler(with_mean=False)),
    ('pca', TruncatedSVD(n_components=2))
])

In [56]:
# Preprocess the full data
pca_sparse_full = clean_and_pca_pipeline.fit(sparse_matrix)

In [52]:
# Split data
sparse_train, sparse_test, train_indices, test_indices = split_pipeline.fit_transform(sparse_matrix)
# Fit the pipeline to training
clean_and_pca_pipeline.fit(sparse_train)
# Transform the training data
pca_sparse_train = clean_and_pca_pipeline.transform(sparse_train)
# Transform the test data
pca_sparse_test = clean_and_pca_pipeline.transform(sparse_test)

c:\Users\nphda\miniconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


In [ ]:
pca_df_full = reindex(pca_sparse_full, indices=train_indices,  columns= column_names, names=row_names)

In [53]:
columns=["PC1", "PC2"]
train_pca_df_test = reindex(pca_sparse_train, indices =train_indices,  columns  = column_names, names = row_names)
test_pca_df_test = reindex(pca_sparse_test, indices =test_indices,  columns  = column_names, names = row_names)

ValueError: Shape of passed values is (1, 1), indices imply (1, 5)

In [ ]:
pca_df_full.to_csv('pca_df_full', index=False)
train_pca_df_test.to_csv('pca_train_df', index=False)
test_pca_df_test.to_csv('pca_test_df', index=False)

## Cluster analysis

### K Means

In [ ]:
kmeans_params = {
    'n_clusters': list(range(1, 10)),
    'init': ['random', 'k-means++'],
    'n_init': [1, 5, 10],
    'max_iter': [300],
    'random_state': [0]
}

choice, k_means_estimator = optimize_and_compare_kmeans(train_without_outliers, kmeans_params)

In [ ]:
find_outliers_in_clusters(train_without_outliers, k_means_estimator)

In [ ]:
label_plot(train_without_outliers, k_means_estimator.labels_, "K Means")

### HDBSCAN

In [ ]:
# Define the parameter grid for HDBSCAN
hdbscan_params = {
    'min_samples': [10, 30, 50, 60, 100],
    'min_cluster_size': [100, 200, 300, 400, 500, 600],
    'cluster_selection_method': ['eom', 'leaf'],
    'metric': ['euclidean', 'manhattan']
}

# Usage example with parameters
result = optimize_and_compare_hdbscan(train_without_outliers, hdbscan_params)

In [ ]:
label_plot(train_without_outliers, result.labels_, "HDBSCAN")

## Validation

### KMeans

### HDBSCAN

# Task 2

In [ ]:
data_path = '../Output/non_zero.parquet'
row_info_path = '../Output/row_names.csv'
column_info_path = '../Output/col_names.csv'
#sparse_matrix = coo_matrix((data['nonzero_elements'], (data['row_indices'], data['col_indices'])))
sparse_matrix_trans, row_names_trans, col_names_trans, row_indices_trans= my_utils.load_data(data_path, row_info_path, column_info_path, transpose = True)

# Preprocess

In [ ]:
def preprocessing(df, columns, row_names):
    # Preprocess the full data
    full = clean_and_pca_pipeline.transform(df)
    # Split data
    train, test, train_indices, test_indices = split_pipeline.fit_transform(df)
    # Fit the pipeline to full
    clean_and_pca_pipeline.fit(full)
    pca_full = clean_and_pca_pipeline.transform(full)
    # Fit the pipeline to training
    clean_and_pca_pipeline.fit(train)
    # Transform the training data
    pca_train = clean_and_pca_pipeline.transform(train)
    # Transform the test data
    pca_test = clean_and_pca_pipeline.transform(test)
    # Reindex 
    columns=["PC1", "PC2"]
    pca_df_full = reindex(pca_full, indices=train_indices,  columns= columns, names=row_names)
    pca_df_train = reindex(pca_train, indices =train_indices,  columns  = columns, names = row_names)
    pca_df_test = reindex(pca_test, indices =test_indices,  columns  = columns, names = row_names)
    # write to csv
    pca_df_full.to_csv('pca_df_full', index=False)
    train_pca_df_test.to_csv('pca_train_df', index=False)
    test_pca_df_test.to_csv('pca_test_df', index=False)
    

In [ ]:
def preprocessing(sparse_matrix_trans, col_names_trans, row_names_trans)

# Cluster analysis

## K Means

In [ ]:
choice, k_means_estimator = optimize_and_compare_kmeans(train_without_outliers, kmeans_params)

In [ ]:
label_plot(train_without_outliers, k_means_estimator.labels_, "K Means")

## HDBSCAN

In [ ]:
# Define the parameter grid for HDBSCAN
hdbscan_params = {
    'min_samples': [10, 30, 50, 60, 100],
    'min_cluster_size': [100, 200, 300, 400, 500, 600],
    'cluster_selection_method': ['eom', 'leaf'],
    'metric': ['euclidean', 'manhattan']
}

# Usage example with parameters
result = optimize_and_compare_hdbscan(train_without_outliers, hdbscan_params)

In [ ]:
label_plot(train_without_outliers, result.labels_, "HDBSCAN")

## Validation

# Cluster Profile

In [ ]:
meta_df = pd.merge(df, metadata, left_on='index', right_on='name', how='left').drop('name', axis=1)
meta_df.head()

In [ ]:
meta_df_demo = meta_df[['cluster', 'patient', 'gender', 'disease', 'celltype']]
meta_df_demo.head()

In [ ]:
meta_df_demo.describe(include=['object']).T

In [ ]:
meta_df_demo.groupby('cluster').describe(include=['object']).T

# Task 3

## Cluster analysis

In [ ]:
t0 = time.time()
brc = Birch(n_clusters=2)
brc.fit(df_t)
t1 = time.time()

In [ ]:
silhouette_score(df_t, brc.labels_)

## Run time analysis